In [ ]:
from sim.simulator import *
from sim.sim_controllers import *
from sim.config import ExperimentConfig
from sim.motor_controllers import *
from utils.path_utils import join_paths
from utils.gui_utils import UserPrompt

In [ ]:
################################ User Input ################################

frames_folder = "data/frames"
output_path = "data/Exp2/"

############################################################################

In [ ]:
reader = FrameReader.create_from_directory(frames_folder, read_format=cv.IMREAD_GRAYSCALE)

In [ ]:
################################ User Input ################################

# create experiment config
experiment_config = ExperimentConfig(
    name="experiment1",
    num_frames=60000,
    frames_per_sec=60,
    orig_resolution=(1600, 1400),
    px_per_mm=90,
    init_position=(1300, 1200),
    comments="",
)

# initialize logging settings
log_config = LogConfig(
    root_folder=output_path,
    save_mic_view=False,
    save_cam_view=False,
    save_err_view=True,
)

# initialize the timing configuration
time_config = TimingConfig(
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.32, 0.32),
    experiment_config=experiment_config,
)

############################################################################

In [ ]:
################################ User Input ################################

model_path = None

if model_path is None:
    model_path = UserPrompt.open_file("Select YOLO model", ("pytorch file", "*.pt"))


yolo_config = YoloConfig(
    model_path=model_path,
    device="cpu",
    pred_kwargs={"imgsz": 384, "conf": 0.1},
)

############################################################################

In [ ]:
# save config files
experiment_config.save_json(join_paths(output_path, "experiment_config.json"))
log_config.save_json(join_paths(output_path, "log_config.json"))
time_config.save_json(join_paths(output_path, "time_config.json"))
yolo_config.save_json(join_paths(output_path, "yolo_config.json"))

In [ ]:
# create YoloController of the simulation that runs in real time and detects worms heap positions from the input frames
yolo_controller = YoloController(time_config, yolo_config)

# wrap the YoloController with a LoggingController to log the simulation data
controller = LoggingController(yolo_controller, log_config)

In [ ]:
# create simulator

motor_controller = SineMotorController(time_config)

sim = Simulator(
    time_config,
    experiment_config,
    controller,
    reader=reader,
    motor_controller=motor_controller,
)

In [ ]:
# run simulation and save log results

sim.run(visualize=False, wait_key=False)